## 詞頻矩陣

In [9]:
import jieba
jieba.load_userdict('userdict.txt')
ary = ['富邦人壽首創保費1萬就能刷卡',
       '富邦人壽不老勇士Go來盃3對3籃賽南北開打',
       '富邦人壽不老勇士Go來盃開打']

In [10]:
corpus = []
for title in ary:
    corpus.append(' '.join(jieba.cut(title)))

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 

X = vectorizer.fit_transform(corpus)

In [12]:
X.shape

(3, 11)

In [13]:
vectorizer.get_feature_names()

['go', '不老', '人壽', '保費', '刷卡', '勇士', '南北', '富邦', '籃賽', '開打', '首創']

In [14]:
X.toarray()

array([[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]], dtype=int64)

## 相似度計算

In [17]:
a = '柯P 大巨蛋 遠雄'
b = '柯P 為了 大巨蛋 一案 找 遠雄 算帳'

#   柯P 大巨蛋 遠雄 為了 一案 找 算帳
# a  1    1    1    0    0   0   0
# b  1    1    1    1    1   1   1

In [25]:
import numpy as np
na = np.array([1,1,1,0,0,0,0])
nb = np.array([1,1,1,1,1,1,1])

np.sqrt(sum((nb - na) ** 2))

2.0

In [27]:
sum(na * nb) / (np.sqrt(sum(na ** 2)) * np.sqrt(sum(nb ** 2)) )

0.6546536707079772

In [23]:
a = '柯P 大巨蛋 遠雄'
b = '柯P 為了 大巨蛋 一案 找 遠雄 算帳 因為 遠雄 在 這案 中 出爾反爾'

#   柯P 大巨蛋 遠雄 為了 一案 找 算帳 因為 在 這案 中 出爾反爾
# a  1    1    1    0    0   0   0  0   0   0   0    0
# b  1    1    2    1    1   1   1  1   1   1   1    1

In [28]:
import numpy as np
na = np.array([1,1,1,0,0,0,0,0,0,0,0,0,0])
nb = np.array([1,1,2,1,1,1,1,1,1,1,1,1,1])

np.sqrt(sum((nb - na) ** 2))

3.3166247903553998

In [29]:
sum(na * nb) / (np.sqrt(sum(na ** 2)) * np.sqrt(sum(nb ** 2)) )

0.57735026918962584

In [31]:
import jieba
jieba.load_userdict('userdict.txt')
ary = ['富邦人壽首創保費1萬就能刷卡',
       '富邦人壽不老勇士Go來盃3對3籃賽南北開打',
       '富邦人壽不老勇士Go來盃開打']
corpus = []
for title in ary:
    corpus.append(' '.join(jieba.cut(title)))
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer() 

X = vectorizer.fit_transform(corpus)
X.toarray()

array([[0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]], dtype=int64)

In [32]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(X)

array([[ 1.        ,  0.31622777,  0.4       ],
       [ 0.31622777,  1.        ,  0.79056942],
       [ 0.4       ,  0.79056942,  1.        ]])

In [33]:
from sklearn.metrics.pairwise import cosine_distances
cosine_distances(X)

array([[ 0.        ,  0.68377223,  0.6       ],
       [ 0.68377223,  0.        ,  0.20943058],
       [ 0.6       ,  0.20943058,  0.        ]])

## 新聞推薦引擎

In [ ]:
import requests
from bs4 import BeautifulSoup
news_ary = []
for page in range(1,30):
    res = requests.get('https://tw.appledaily.com/new/realtime/{}'.format(page))
    soup = BeautifulSoup(res.text, 'html.parser')
    for rec in soup.select('.rtddt a'):
        dic = {}
        dic['link'] = rec['href']
        res2 = requests.get(dic['link'])
        soup2 = BeautifulSoup(res2.text, 'html.parser')
        dic['category'] = soup2.select('.current')[2].text
        dic['title']    = soup2.select_one('h1').text
        dic['content']  = soup2.select_one('.ndArticle_margin p').text
        news_ary.append(dic)
    print(page)

1
2
3
4
5
6
7


In [16]:
synonym_dic = {
    '柯P': '柯文哲'
}
s = '柯P為了大巨蛋一案找遠雄算帳'
' '.join([synonym_dic.get(ele, ele) for ele in jieba.cut(s)])


'柯文哲 為 了 大巨蛋 一案 找遠 雄算帳'